In [8]:
#############################################################
#                     Naver Webtton Crwaling                #
#############################################################

import requests
import csv
from bs4 import BeautifulSoup
import re
import json
import time
import argparse
import requests
import pathlib2
import openpyxl
from urllib.parse import urlparse, parse_qs
import pymysql

naver_comment_db = pymysql.connect(
    user='root', 
    passwd='alsdk0306', 
    port=3306,
    host='127.0.0.1', 
    db='naver_comment_db', 
    charset='utf8'
)

cursor = naver_comment_db.cursor(pymysql.cursors.DictCursor)

def get_commentList(title, titleId, episode_no, comment_page):
    ajax_link = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon' + \
                '&pool=cbox3&_callback=jQuery112409768039369695578_1588773317723&lang=ko&country=KR&objectId=' + \
                titleId + '_' + episode_no + '&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=' + str(comment_page) + \
                '&refresh=true&sort=new&cleanbotGrade=2&_=1588773317725'
    custom_header = {
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=' + titleId + '&no=' + episode_no,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'
    }
    req = requests.get(ajax_link, headers=custom_header).text
    json_req = json.loads(req[req.find("success") - 2:len(req) - 2])
    commentList = json_req['result']['commentList']
    return commentList

def save_comments(title, titleId, episode_no):
    
    comment_page = 1
    all_comments_info= []
    cnt=0
    num=0

    while True:
        cnt+=1
        commentList = get_commentList(title, titleId, episode_no, comment_page)
        if commentList:
            for comment in commentList:
              num+=1;
              all_comments_info=[cnt]
              all_comments_info.append(title)
              all_comments_info.append(episode_no)
              all_comments_info.append(comment['maskedUserId'])
              all_comments_info.append(comment['contents'].replace("\n",""))
              print(str(num)+'번째 ',title,episode_no,comment['maskedUserId'],comment['contents'].replace("\n",""))

              
              sql = """INSERT INTO naver_webtoon_comment (title, user_id, comment) VALUES( '""" + title + """', '""" + comment['maskedUserId'] + """', '""" + comment['contents'].replace("\n","") +"""')""" 
              print(sql) 
              cursor.execute(sql)
        else:
            return
        comment_page = comment_page + 1



url_episode = 'https://comic.naver.com/webtoon/genre.nhn'
url_omnibus = 'https://comic.naver.com/webtoon/genre.nhn?genre=omnibus'
url_story = 'https://comic.naver.com/webtoon/genre.nhn?genre=story'

url_list = [url_episode, url_omnibus, url_story]
title_list = []
for i in url_list:
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html.parser')
    temp = soup.find_all('dl')
    for j in temp:
        title_list.append(j)

cnt=0
for a in title_list:
    cnt+=1
    print(cnt, "번째 웹툰 크롤링 중...")
    webtoon_info = [cnt]
    webtoon_info.append(a.find('a').get('title'))
    print("제목 : ", a.find('a').get('title'))
    webtoon_info.append(a.find('dd').get_text()[1:len(a.find('dd').get_text())-1])
    print("작가 : ", a.find('dd').get_text()[1:len(a.find('dd').get_text())-1])
   
    webtoon_url =  "https://comic.naver.com" + a.find('a').get('href')
    response = requests.get(webtoon_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(webtoon_url)

    webtoon_round_url=soup.find('td',class_='title').find('a').get('href')
    print(soup.find('td',class_='title').find('a').get('href'))

    #id, 회차정보, 요일
    tId=webtoon_round_url.split('titleId=')[1].split('&')[0]
    tRound=webtoon_round_url.split('no=')[1].split('&')[0]
    tWeekday=webtoon_round_url.split('weekday=')[1].split('&')[0]
    webtoon_info.append(tId)
    webtoon_info.append(tRound)
    webtoon_info.append(tWeekday)

    print("아이디: ",tId)
    print("회차정보: ", tRound)
    print("요일: ",tWeekday)

    for i in range(int(tRound)):
      # https://comic.naver.com/webtoon/detail.nhn?titleId=733766&no=56&weekday=mon 형식
      webtoon_comment_url="https://comic.naver.com/webtoon/detail.nhn?"+ "titleId="+tId+"&no="+str(i)+"&weekday="+tWeekday
      save_comments(a.find('a').get_text(),tId,str(i))
      
naver_comment_db.commit()
naver_comment_db.close()
print(cnt, "개 웹툰 크롤링 종료")



OperationalError: ignored

In [5]:
pip install pathlib2